## Notebook description

This notebook contains codes for:
- Part 1: Class imbalance techniques - SMOTE, ADASYN, ENN, SMOTEENN, SMOTETomek
- Part 2: Applying adaboost on the ENN-handled model
- Part 3: Hyperparameter tune it

In [33]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.layers import Dropout
from tensorflow.keras import optimizers

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import metrics

from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek 

import pandas as pd
import numpy as np

from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import GaussianNB

from catboost import CatBoostClassifier, Pool
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier

In [34]:
#adjusting the working directory to correspond to individual file path 
import os
path = os.getcwd()

temp_list = []
for ch in path:
    if ch != "\\":
        temp_list.append(ch)
    else:
        temp_list.append("/")

path_2 = ''

for item in temp_list:
    path_2+= item

print(path_2)

/content


In [35]:
#Load the dataset
'''df = pd.read_csv(path_2+ "/encoded_df.csv" )'''

from google.colab import drive
drive.mount('/content/drive/')
#Load data

df = pd.read_csv("/content/drive/My Drive/IS460/18 November compilation for final submission/encoded_df.csv")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [36]:
#view the first 5 rows of the dataframe 
df.head()

,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,1,445412.0,1,709.0,1167493.0,8.0,1,5,5214.74,17.2,6.0,1.0,228190.0,416746.0,1.0,0.0
1,1,262328.0,1,724.0,1174162.0,1.0,1,3,33295.98,21.1,35.0,0.0,229976.0,850784.0,0.0,0.0
2,1,99999999.0,1,741.0,2231892.0,8.0,2,3,29200.53,14.9,18.0,1.0,297996.0,750090.0,0.0,0.0
3,1,347666.0,0,721.0,806949.0,3.0,2,3,8741.90,12.0,9.0,0.0,256329.0,386958.0,0.0,0.0
4,1,176220.0,1,724.0,1174162.0,5.0,3,3,20639.70,6.1,15.0,0.0,253460.0,427174.0,0.0,0.0


In [37]:
# Shape of df
df.shape

(92814, 16)

### Part 1 - Class imbalance techniques

In [38]:
#Dropping 5 columns which have VIF > 5
df1 = df.drop(['Annual Income', 'Number of Credit Problems', 
                     'Number of Open Accounts', 'Years of Credit History', 
                     'Current Credit Balance'], axis = 1)

#dropping rows with 99999999.0 as value under Current Loan Amount column
df1 = df1[df1["Current Loan Amount"]!=99999999.0]

In [39]:
df1.shape

(82389, 11)

In [40]:
#Set the target and features column
y= df1["Loan Status"]
x = df1.drop(["Loan Status"],axis=1)

In [41]:
# Split dataset into 80% train, 20% test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, 
                                                          stratify = y, random_state = 42)

#Standardising with Robust Scaler-- robust to outliers
ro_scaler = RobustScaler()
x_train = ro_scaler.fit_transform(x_train)
x_test = ro_scaler.transform(x_test)

ENN

In [42]:
#ENN
y= df1["Loan Status"]
x = df1.drop(["Loan Status"],axis=1)
# Split dataset into 80% train, 20% test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, 
                                                          stratify = y, random_state = 42)

#Standardising with Robust Scaler-- robust to outliers
ro_scaler = RobustScaler()
x_train = ro_scaler.fit_transform(x_train)
x_test = ro_scaler.transform(x_test)

print("Before resampling: ", Counter(y_train))

enn = EditedNearestNeighbours()

x_resampled, y_resampled = enn.fit_resample(x_train, y_train)

print("After resampling: ", Counter(y_resampled))


Before resampling:  Counter({1: 48677, 0: 17234})
After resampling:  Counter({1: 25614, 0: 17234})


Models

In [43]:
DTree=DecisionTreeClassifier(criterion="gini",max_depth=5, random_state = 42)
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2 )
LR = LogisticRegression(random_state = 42)
Naive = GaussianNB()
cb = CatBoostClassifier(random_state=42,  eval_metric = "F1")
gb = GradientBoostingClassifier(n_estimators=500,
                                       learning_rate=0.1,
                                       max_depth=3,
                                       subsample=0.5,
                                       validation_fraction=0.1,
                                       n_iter_no_change=20,
                                       max_features='log2',
                                       verbose=1)
xgb = XGBClassifier(seed = 42)
lgb = lgb.LGBMClassifier()
svm = LinearSVC(dual = False, random_state = 42)
forest = RandomForestClassifier(random_state=42)

Voting Classfier

In [44]:
import time
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score, KFold
estimators = []
# k-nearest neighbors vote (classification and regression)
estimators.append(('knc', KNN))
# Support Vector Classification (supervised)
estimators.append(('svc', svm))
# supervised; classification or regression
estimators.append(('dtc', DTree))
# ensemble learning method (uses multiple learning algorithms) for classification, regression
estimators.append(('rfc', forest))
# classification
estimators.append(('abc', AdaBoostClassifier()))
# regression and classification; ensemble of weak prediction models, typically decision trees
estimators.append(('gbc', gb))
# optimized distributed gradient boosting; parallel tree boosting
estimators.append(('xgb', xgb))
# 
#estimators.append(('NN', MLPClassifier(max_iter = 1000)))
# 
estimators.append(('nbc', Naive))
# 
estimators.append(('lr', LR))


t0 = time.time()
# Soft Voting/Majority Rule classifier for unfitted estimators

# uses different portions of the data to test and train a model on different iterations;
# uses where the goal is prediction, and wants to estimate how accurate a predictive model will perform
'''results = cross_val_score(ensemble1, x_res, y_res, cv=5, scoring='f1_macro')
t1 = time.time()
print(('cross_val_score time: %.2fs' % (t1 - t0)).lstrip('0'))
print(results.mean())'''


t0 = time.time()
#ensemble1.fit(x_res, y_res)
ensemble = VotingClassifier(estimators)
ensemble.fit(x_resampled, y_resampled)
t1 = time.time()
print(('Ensemble1 time: %.2fs' % (t1 - t0)).lstrip('0'))
# Predictions for X_test are compared with y_test; either accuracy (classifiers) or R² score (regression)
print("Accuracy on training set: {:.3f}".format(ensemble.score(x_resampled, y_resampled)))
print("Accuracy on validation set: {:.3f}".format(ensemble.score(x_test, y_test)))

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.3216           0.0259           12.07s
         2           1.2986           0.0255           11.85s
         3           1.2784           0.0163           11.91s
         4           1.2677           0.0094           12.29s
         5           1.2603           0.0072           11.94s
         6           1.2506           0.0136           12.50s
         7           1.2437           0.0018           12.18s
         8           1.2364           0.0123           12.02s
         9           1.2241           0.0107           12.51s
        10           1.2164           0.0088           12.36s
        20           1.1673           0.0043           20.45s
        30           1.1395           0.0005           17.89s
        40           1.1331           0.0007           15.88s
        50           1.1174           0.0012           14.56s
        60           1.1099           0.0004           13.53s
       

In [45]:
y_pred = ensemble.predict(x_test)

classReport = classification_report( y_test,y_pred)
print(classReport)

              precision    recall  f1-score   support

           0       0.56      0.36      0.44      4308
           1       0.80      0.90      0.85     12170

    accuracy                           0.76     16478
   macro avg       0.68      0.63      0.64     16478
weighted avg       0.74      0.76      0.74     16478

